In [ ]:
%matplotlib inline

# Multi-point computations using namespaces.


In [ ]:
from __future__ import annotations

from gemseo import create_design_space
from gemseo import create_discipline
from gemseo import create_scenario

## About namespaces

Namespaces are prefixes to input or output names of the disciplines.
The name of a variable can be prefixed by the namespace and a separator, ":" by default.
This allows to control the coupling of disciplines, or to duplicate a discipline
in a process.



##Instantiate the disciplines

We instantiate twice the same discipline, they will be configured with the namespaces
to compute different outputs. Some inputs will be shared, without namespace, and others
will have different namespaces.




In [ ]:
disciplines = create_discipline(["SobieskiMission", "SobieskiMission"])

## Setting the namespaces

In the SSBJ test case, the `y_34` vector is the engine Specific Fuel Consumption (SFC).
Let's say that we want to compute the average range of the aircraft, operating at
two different operating conditions (`oc1` and `oc2`), which leads to two different SFC.
We add the specific namespaces to the SFC (`y_34`) of the disciplines,
and to the output range (`y_4`)
The other inputs of the discipline remain the same, so they share their values.



In [ ]:
disciplines[0].input_grammar.add_namespace("y_34", "oc1")
disciplines[1].input_grammar.add_namespace("y_34", "oc2")

disciplines[0].output_grammar.add_namespace("y_4", "oc1")
disciplines[1].output_grammar.add_namespace("y_4", "oc2")

## Averaging the outputs

Then we need to create a specific discipline to average the outputs.
We use the LinearCombination discipline for that.
Note that the namespaces can be set by the input name directly.



In [ ]:
disciplines.append(
    create_discipline(
        "LinearCombination",
        input_names=["oc1:y_4", "oc2:y_4"],
        output_name="average_y_4",
        input_coefficients={"oc1:y_4": 0.5, "oc2:y_4": 0.5},
    )
)

## Build, execute and post-process the scenario

In the design space, we add the two variables that are namespaced, and
simulate several SFCs. The other variables remain at their default input's values.



In [ ]:
design_space = create_design_space()
design_space.add_variable("oc1:y_34", lower_bound=0.8, upper_bound=2.0)
design_space.add_variable("oc2:y_34", lower_bound=0.8, upper_bound=2.0)

### Instantiate the scenario

The objective is the averaged range.



In [ ]:
scenario = create_scenario(
    disciplines,
    formulation_name="MDF",
    objective_name="average_y_4",
    design_space=design_space,
    maximize_objective=True,
    scenario_type="DOE",
)

### Visualize the XDSM

The XDSM shows well the averaging proces and the duplication of the disciplines
as well as the data handling



In [ ]:
scenario.xdsmize(save_html=False)

scenario.execute(n_samples=20, algo_name="LHS")

### Plot the optimization history view




In [ ]:
scenario.post_process(post_name="OptHistoryView", save=False, show=True)